In [1]:
import io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats
import scipy.special
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook')
from sklearn.model_selection import train_test_split
import os

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score

from sklearn import metrics
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import TomekLinks


from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE

##### from collections import Counter
from sklearn.datasets import make_classification
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

from imblearn.ensemble import EasyEnsembleClassifier

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


import scipy.stats as st
import math

seed=45

## Importing Training Set

In [2]:
data=pd.read_csv('/Users/ibrain/Downloads/Cognitive Neuroscience Work/Cognition and Lifestyle R Coding/Cognition_Lifestyle/Imbalanced Data/training_set.csv')
data=data.iloc[:,1:]
data.head()
print(data.shape)

X = data.drop(columns='CIMEMLOS')
Y = data['CIMEMLOS']

X_orig=X
Y_orig=Y

(16809, 18)


In [3]:
X.head()

,SEXVAR,X.AGEG5YR,X.BMI5,EXERANY2,SLEPTIM1,ADDEPEV3,DIABETE4,CVDSTRK3,smok_everyday,smok_somday,smok_no,SES,White,Black,Hispanic,Multi,Other
0,0,10,28.29,1,7,0,1,0,0,0,1,11,1,0,0,0,0
1,1,11,37.97,0,6,0,1,0,0,0,1,5,1,0,0,0,0
2,0,9,19.84,1,8,0,0,0,1,0,0,14,1,0,0,0,0
3,0,7,21.11,0,4,1,1,0,0,0,1,6,1,0,0,0,0
4,1,11,20.53,0,4,0,0,0,1,0,0,12,1,0,0,0,0


## Importing Evaluation Set

In [4]:
eval_set=pd.read_csv('/Users/ibrain/Downloads/Cognitive Neuroscience Work/Cognition and Lifestyle R Coding/Cognition_Lifestyle/Imbalanced Data/evaluation_set.csv')
eval_set=eval_set.iloc[:,1:]
eval_set.head()
print(eval_set.shape)

eval_X = eval_set.drop(columns='CIMEMLOS')
eval_Y = eval_set['CIMEMLOS']

(2401, 18)


## Importing the Test Set

In [5]:
test_data=pd.read_csv('/Users/ibrain/Downloads/Cognitive Neuroscience Work/Cognition and Lifestyle R Coding/Cognition_Lifestyle/Imbalanced Data/testing_set.csv')
test_data=test_data.iloc[:,1:]
test_data.head()
print(test_data.shape)

test_X = test_data.drop(columns='CIMEMLOS')
test_Y = test_data['CIMEMLOS']

(4802, 18)


## Cluster Centroids + ADASYN

In [6]:
## CC
print('Original dataset shape %s' % Counter(Y))
cc = ClusterCentroids(estimator=MiniBatchKMeans(n_init=1, random_state=seed), random_state=seed, voting="soft",
                     sampling_strategy={0:12000,1:1805})
X_res, y_res = cc.fit_resample(X, Y)
print('Resampled dataset shape %s' % Counter(y_res))

#ADASYN on the dataset returned by CC
ada = ADASYN(random_state=42)
print(sorted(Counter(y_res).items()))
X_res, y_res = ada.fit_resample(X_res, y_res)
print(sorted(Counter(y_res).items()))


Original dataset shape Counter({0: 15004, 1: 1805})
Resampled dataset shape Counter({0: 12000, 1: 1805})
[(0, 12000), (1, 1805)]
[(0, 12000), (1, 12087)]


In [7]:
#Import library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
    
seed =45

# two methods are used : BaggingClassifier and Adaboost Classifier. You can choose any by commenting the other.

# bbc = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
#                         max_samples=1.0,
#                         n_estimators=500,
#                         random_state=seed)

bbc = AdaBoostClassifier(n_estimators=500,random_state=seed)
bbc_original=bbc
df=pd.DataFrame(columns = ['Xtr','ytr', 'Spec','Prec', 'Recall', 'F1', 'BAC', 'AUC',
                          'TT', 'VT'])

kf = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
X=X_res
Y=y_res
print(sorted(Counter(y_res).items()))

for train_index,test_index in kf.split(X, Y):
    #print('{} of KFold {}'.format(i,kf.n_splits))
    xtr,xvl = X.iloc[train_index],X.iloc[test_index]
    #print(Counter(yvl))
    ytr,yvl = Y.iloc[train_index],Y.iloc[test_index]
    
    start1 = time.time()
    bbc.fit(xtr,ytr)
    end1=time.time()
    
    start2 = time.time()
    y_pred=bbc.predict(xvl)
    end2 = time.time()
    
    
    classification_metrics=classification_report(yvl, y_pred,output_dict=True)
    #print(confusion_matrix(yvl,y_pred))
    bac=balanced_accuracy_score(yvl, y_pred)
    temp_dict=classification_metrics.get('1')

    y_pred_proba=bbc.predict_proba(xvl)[::,1]

    # Calculation of AUC
    auc=metrics.roc_auc_score(yvl, y_pred_proba)
    
    new_row = pd.Series({'Xtr': len(xtr), 'ytr': ytr.value_counts()[1],
                         'Spec': classification_metrics.get('0').get('recall'),
                         'Prec': temp_dict.get('precision'),
                         'Recall': temp_dict.get('recall'),
                         'F1': temp_dict.get('f1-score'),
                         'BAC': bac, 'AUC': auc,
                         'TT': end1 - start1, 'VT': end2 - start2})
    df=pd.concat([df, new_row.to_frame().T], ignore_index=True)


print("Cross Validation Results printed....")

mean_eval_array=[]

for name in df.columns[2:]:
    print('\n------------------------',name,'---------------------------\n')
    #plt.hist(df[name])
    #plt.show()

    print(name, "Mean=" , round(df[name].mean(),2))

    t=st.t.interval(df=len(df)-1,alpha=0.05,
              loc=np.mean(df[name]),
              scale=st.sem(df[name]))


    print( round(df[name].mean(),2), "[", round(t[0],2),',', round(t[1],2),"]", round(st.sem(df[name]),2))
    mean_eval_array.append(round(df[name].mean(),2))
    

# Using the model on the validation set

print("\n\nValidation Results printed....\n")
print(len(X_res), y_res.value_counts(), len(eval_X), eval_Y.value_counts())

bbc = bbc_original
bbc.fit(X_res, y_res)
y_pred=bbc.predict(eval_X)

print(confusion_matrix(eval_Y, y_pred))
print(classification_report(eval_Y, y_pred))
y_pred_proba=bbc.predict_proba(eval_X)[::,1]

    # Calculation of AUC
auc=metrics.roc_auc_score(eval_Y, y_pred_proba)
print(auc)

[(0, 12000), (1, 12087)]
Cross Validation Results printed....

------------------------ Spec ---------------------------

Spec Mean= 0.83
0.83 [ 0.83 , 0.83 ] 0.0

------------------------ Prec ---------------------------

Prec Mean= 0.8
0.8 [ 0.8 , 0.8 ] 0.0

------------------------ Recall ---------------------------

Recall Mean= 0.67
0.67 [ 0.67 , 0.67 ] 0.0

------------------------ F1 ---------------------------

F1 Mean= 0.73
0.73 [ 0.73 , 0.73 ] 0.0

------------------------ BAC ---------------------------

BAC Mean= 0.75
0.75 [ 0.75 , 0.75 ] 0.0

------------------------ AUC ---------------------------

AUC Mean= 0.84
0.84 [ 0.84 , 0.84 ] 0.0

------------------------ TT ---------------------------

TT Mean= 3.78
3.78 [ 3.78 , 3.78 ] 0.04

------------------------ VT ---------------------------

VT Mean= 0.1
0.1 [ 0.1 , 0.1 ] 0.0


Validation Results printed....

24087 1    12087
0    12000
Name: CIMEMLOS, dtype: int64 2401 0    2120
1     281
Name: CIMEMLOS, dtype: int64
[[17

## Combine Training and Evaluation Set for Complete Training

In [8]:
train_eval=[data, eval_set]
complete_train=pd.concat(train_eval)
print(len(complete_train))
complete_train_X = complete_train.drop(columns='CIMEMLOS')
complete_train_Y = complete_train['CIMEMLOS']
print(complete_train_Y.value_counts())

19210
0    17124
1     2086
Name: CIMEMLOS, dtype: int64


## Apply CC + ADASYN to the entire dataset

In [9]:
##CC
print('Original dataset shape %s' % Counter(complete_train_Y))
cc = ClusterCentroids(estimator=MiniBatchKMeans(n_init=1, random_state=seed), random_state=seed, voting="soft",
                     sampling_strategy={0:15000,1:1805})
X_res, y_res = cc.fit_resample(complete_train_X, complete_train_Y)
print('Resampled dataset shape %s' % Counter(y_res))


#ADASYN
ada = ADASYN(random_state=42)
print(sorted(Counter(y_res).items()))
X_res, y_res = ada.fit_resample(X_res, y_res)
print(sorted(Counter(y_res).items()))


Original dataset shape Counter({0: 17124, 1: 2086})
Resampled dataset shape Counter({0: 15000, 1: 1805})
[(0, 15000), (1, 1805)]
[(0, 15000), (1, 15637)]


## Checking the Classifier

In [10]:
bbc_original

AdaBoostClassifier(n_estimators=500, random_state=45)

In [11]:
bbc = bbc_original
bbc.fit(X_res, y_res)
y_pred=bbc.predict(test_X)

print(confusion_matrix(test_Y, y_pred))
print(classification_report(test_Y, y_pred))
y_pred_proba=bbc.predict_proba(test_X)[::,1]

# Calculation of AUC
auc=metrics.roc_auc_score(test_Y, y_pred_proba)
print(auc)

[[3609  688]
 [ 382  123]]
              precision    recall  f1-score   support

           0       0.90      0.84      0.87      4297
           1       0.15      0.24      0.19       505

    accuracy                           0.78      4802
   macro avg       0.53      0.54      0.53      4802
weighted avg       0.83      0.78      0.80      4802

0.5732214738811559
